# 5.1 层和块

In [1]:
# 多层感知机生成一个网络
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0820,  0.0323,  0.0058,  0.2228,  0.0309, -0.0699, -0.0925,  0.1462,
         -0.0402, -0.0232],
        [ 0.1388,  0.1081,  0.0590,  0.0582,  0.0700, -0.1320, -0.1109,  0.1666,
         -0.0869, -0.0179]], grad_fn=<AddmmBackward0>)

In [2]:
# 自定义块
class MLP(nn.Module):
  # 用模型参数声明层这里，我们声明两个全连接层
  def __init__(self):
    # 调用MLP的父类Module的构造函数来执行必要的初始化
    # 这样在类实例化时也可以指定其他函数参数，例如模型参数params
    super().__init__()
    self.hidden = nn.Linear(20, 256) # 隐藏层
    self.out = nn.Linear(256, 10) # 输出层

  # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
  def forward(self, X):
    return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[ 0.0007, -0.0674,  0.1985,  0.0114,  0.1061, -0.0501,  0.0435,  0.0631,
          0.1987, -0.1977],
        [ 0.0736, -0.0028,  0.1999, -0.0065,  0.0886, -0.0067, -0.0093,  0.0259,
          0.1512, -0.0530]], grad_fn=<AddmmBackward0>)

In [3]:
# 顺序块
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for idx, module in enumerate(args):
      # 这里，module是Module子类的一个实例我们把它保存在Module类的成员
      # 变量_modules中，_moduels的类型是OrderedDict
      self._modules[str(idx)] = module

  def forward(self, X):
    # OrderedDict 保证了按照成员添加的顺序遍历它们
    for block in self._modules.values():
      X = block(X)
    return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0016, -0.0798, -0.2800, -0.0268, -0.1793, -0.0567,  0.1595,  0.0186,
          0.0049,  0.1489],
        [ 0.0190, -0.1754, -0.2609,  0.0243, -0.1618,  0.1256,  0.1446,  0.1194,
          0.0219,  0.1300]], grad_fn=<AddmmBackward0>)

In [4]:
# 在前向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()
    # 不计算梯度的随机权重参数，因此其在训练期间保持不变
    self.rand_weight = torch.rand((20, 20), requires_grad=False)
    self.Linear = nn.Linear(20, 20)

  def forward(self, X):
    X = self.Linear(X)
    # 使用创建的常量参数以及relu函数和mm函数
    X = F.relu(torch.matmul(X, self.rand_weight) + 1)
    # 复用全连接层相当于两个全连接层共享参数
    X = self.Linear(X)
    # 控制流
    while X.abs().sum() > 1:
      X /= 2
    return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0826, grad_fn=<SumBackward0>)

In [5]:
# 嵌套块
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                             nn.Linear(64, 32), nn.ReLU())
    self.Linear = nn.Linear(32, 16)

  def forward(self, X):
    return self.Linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=32, bias=True)
      (3): ReLU()
    )
    (Linear): Linear(in_features=32, out_features=16, bias=True)
  )
  (1): Linear(in_features=16, out_features=20, bias=True)
  (2): FixedHiddenMLP(
    (Linear): Linear(in_features=20, out_features=20, bias=True)
  )
)

# 5.2 参数管理

In [7]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
X
net(X)

tensor([[0.1296],
        [0.1168]], grad_fn=<AddmmBackward0>)

In [9]:
# 参数访问
print(net[2].state_dict()) # 访问第二个全连接层的参数

OrderedDict([('weight', tensor([[ 0.0686,  0.0855, -0.2551, -0.0684,  0.3075,  0.0167,  0.0675, -0.2693]])), ('bias', tensor([0.2112]))])


In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2112], requires_grad=True)
tensor([0.2112])


In [11]:
net[2].weight.grad == None

True

In [13]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()]) # 第一个全连接层的参数
print(*[(name, param.shape) for name, param in net.named_parameters()]) # 所有全连接层的参数

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [14]:
net.state_dict()['2.bias'].data

tensor([0.2112])

In [17]:
def block1():
  return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                       nn.Linear(8, 4), nn.ReLU())

def block2():
  net = nn.Sequential()
  for i in range(4):
    # 在这里嵌套
    net.add_module(f'block {i}', block1())
  return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.5279],
        [-0.5279]], grad_fn=<AddmmBackward0>)

In [18]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [19]:
rgnet[0][1][0].bias.data

tensor([-0.2731, -0.0847, -0.0257, -0.4854,  0.3559,  0.3389,  0.1234, -0.2842])

In [21]:
# 参数初始化
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, mean=0, std=0.01) # 初始化权重
    nn.init.zeros_(m.bias) # 初始化偏置
net.apply(init_normal) # 对net的所有layer应用该初始化
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0057,  0.0148,  0.0113,  0.0098]), tensor(0.))

In [22]:
def init_constant(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 1)
    nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [24]:
def init_xvaier(m):
  if type(m) == nn.Linear:
    nn.init.xavier_uniform_(m.weight)
def init42(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 42)

net[0].apply(init_xvaier)
net[2].apply(init42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1637,  0.6176,  0.5317, -0.5821])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [27]:
# 自定义初始化
def my_init(m):
  if type(m) == nn.Linear:
    print('Init', *[(name, param.shape) for name, param in m.named_parameters()][0])
    nn.init.uniform_(m.weight, -10, 10)
    m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])
Init weight torch.Size([8, 4])
Init weight torch.Size([4, 8])
Init weight torch.Size([8, 4])
Init weight torch.Size([4, 8])
Init weight torch.Size([8, 4])
Init weight torch.Size([4, 8])
Init weight torch.Size([8, 4])
Init weight torch.Size([4, 8])


tensor([[7.9150, -0.0000, -0.0000, -0.0000],
        [-0.0000, -0.0000, 7.2166, 5.8325]], grad_fn=<SliceBackward0>)

In [29]:
# 共享参数
# 我们需要给共享层提供一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 5.4 自定义层

In [31]:
# 不带参数的层
import torch
from torch import nn
from torch.nn import functional as F

class CenteredLayer(nn.Module):
  def __init__(self):
    super().__init__()
  def forward(self, X):
    return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [33]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(4.6566e-10, grad_fn=<MeanBackward0>)

In [36]:
# 带参数的层
class MyLinear(nn.Module):
  def __init__(self, in_units, units):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(in_units, units))
    self.bias = nn.Parameter(torch.randn(units))
  def forward(self, X):
    linear = torch.matmul(X, self.weight.data) + self.bias.data
    return F.relu(linear)

linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.5458,  0.0322,  0.3133],
        [ 1.3783,  0.0679,  0.0917],
        [-1.1594, -0.2625, -0.9004],
        [ 0.0118,  1.3170, -0.4302],
        [-1.5028,  0.5110, -0.0635]], requires_grad=True)

In [38]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.0402],
        [0.0000, 0.0000, 0.5261]])

In [39]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.9383],
        [2.6671]])

# 5.5 读写文件

In [40]:
# 加载和保存张量
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [41]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [42]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
x2, y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [43]:
mydict = {'x':x, 'y':y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [44]:
# 加载和保存模型参数
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20, 256)
    self.output = nn.Linear(256, 10)

  def forward(self, X):
    return self.output(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [45]:
torch.save(net.state_dict(), 'mlp.params')
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [46]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# 5.6 GPU

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found
